In [1]:
import networkx as nx;
import matplotlib.pyplot as plt
import pandas as pd;
import gurobipy as gp;
from gurobipy import GRB;
import csv;
import sys;
import time;
from datetime import datetime;
import math;
import random;
import itertools;

In [2]:
# Master


def Master(Z, bud, G, tau, s, t, it, t_lim):
    
    M = A + 1;
    My = M;
    Md = M;
    Mt = M;
    
    model = gp.Model("Master-Problem");
    
    model.setParam('TimeLimit', t_lim);
    
    
    # IntegralityFocus parameter to deal with binary variables issues like in bigM constraints
    model.setParam("IntegralityFocus",1);
    #model.setParam("IntFeasTol",1e-7);
    #model.setParam("FeasibilityTol",1e-7);
    #model.setParam("OptimalityTol",1e-7);
    model.setParam("NumericFocus",2);
    #model.setParam("MIPFocus",3);

    
    # Variables (Upper Level)
    eta = model.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    gamma = model.addVars(G.edges, vtype=GRB.BINARY);
    xBar = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    lamb = model.addVars(tau, vtype=GRB.BINARY); 
    
    x = {};
    y = {};
    alpha = {};
    theta = {};
    pi = {};
    delta = {};
    v = {};
    muTau = {};
    mu = {};
    
    flow = 0;
    
   
    
    # Constraints ("Upper Level")
    model.addConstr(gp.quicksum(gamma[i,j]*G.edges[i,j]['cost'] for i,j in G.edges) <= budget[b]);  
    
    model.addConstrs(gp.quicksum(xBar[j,i] for i in G.successors(j))
                 - gp.quicksum(xBar[i,j] for i in G.predecessors(j)) == 0 for j in G.nodes);       
    
    model.addConstr(xBar[t,s] == gp.quicksum(pow(2, tau[u])*lamb[u] for u in tau));
    
    for i,j in G.edges:
        model.addConstr(xBar[i,j] - (1-gamma[i,j])*G.edges[i,j]['capacity'] <= 0);  

    
    
    # Variables ("Lower Level")
    for n in range(len(Z)):
        x[n] = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
        y[n] = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY); 
        alpha[n] = model.addVars(G.nodes, vtype=GRB.CONTINUOUS, lb=-GRB.INFINITY, ub = GRB.INFINITY); 
        theta[n] = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY); 
        pi[n] = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = -My, ub=0);
        delta[n] = model.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub=0);                               
        v[n] = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY); 
        muTau[n] = model.addVars(tau, vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = 0);
        mu[n] = model.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = 0);                      
        
        
    # Constraints ("Lower Level")
        model.addConstrs(gp.quicksum(x[n][j,i] for i in G.successors(j)) 
                         - gp.quicksum(x[n][i,j] for i in G.predecessors(j)) == 0 for j in G.nodes);           
    
        for i,j in G.edges:
            model.addConstr(x[n][i,j] - (1-gamma[i,j])*G.edges[i,j]['capacity'] <= 0);                      
            model.addConstr(v[n][i,j] <= theta[n][i,j]);                                                   
            model.addConstr(v[n][i,j] >= theta[n][i,j] - Mt*gamma[i,j]);
            model.addConstr(v[n][i,j] <= Mt*(1-gamma[i,j]));
            if G.edges[i,j]['special'] == 1:
                model.addConstr(x[n][i,j] + y[n][i,j] >= (1/M)*Z[n][i,j]*G.edges[i,j]['special']);                                         
                model.addConstr(alpha[n][i] - alpha[n][j] + theta[n][i,j] + pi[n][i,j]*G.edges[i,j]['special'] >= 0);               
            elif i!=t:
                model.addConstr(alpha[n][i] - alpha[n][j] + theta[n][i,j] >= 0);                           
            
    
        model.addConstr(x[n][t,s] >= xBar[t,s]);                                                          
    
        model.addConstr(alpha[n][t] - alpha[n][s] + theta[n][t,s] + delta[n] >= 0);                                    
    
        model.addConstr(mu[n] == gp.quicksum(pow(2, tau[u])*muTau[n][u] for u in tau));
        
        for u in tau:
            model.addConstr(muTau[n][u] >= delta[n]);
            model.addConstr(muTau[n][u] >= -Md*lamb[u]);
            model.addConstr(muTau[n][u] <= delta[n] - Md*(lamb[u] - 1));
    
        model.addConstr(eta >= gp.quicksum(Z[n][i,j]*G.edges[i,j]['special'] for i,j in G.edges) +
                    gp.quicksum(G.edges[i,j]['capacity']*v[n][i,j] for i,j in G.edges) +
                    (1/M)*gp.quicksum(Z[n][i,j]*G.edges[i,j]['special']*pi[n][i,j] for i,j in G.edges) + mu[n]); 
    
        model.addConstr(-My*gp.quicksum(y[n][i,j]*G.edges[i,j]['special'] for i,j in G.edges) 
                    >= gp.quicksum(G.edges[i,j]['capacity']*v[n][i,j] for i,j in G.edges) + 
                    (1/M)*gp.quicksum(Z[n][i,j]*G.edges[i,j]['special']*pi[n][i,j] for i,j in G.edges) + mu[n]); 
    
    
    
    
    model.setObjective(eta, GRB.MINIMIZE);
    model.update();
    model.setParam("OutputFlag", 0);
    model.optimize();
    
    if model.status == GRB.status.OPTIMAL:
        LB = model.objVal;  
        print('Iteration: %g' %it); 
        print('LB (Master): %g' %LB);
        for i,j in G.edges:
            gammaStar[i,j] = gamma[i,j].x;
            if gamma[i,j].x > 0:
                print('Interdict: ',i,j);
                 
        print('xBar_ts', 'flow :', xBar[t,s].x);
        print('x_ts', 'flow :', x[n][t,s].x);
        flow = x[n][t,s].x;
            
    
    elif model.status == GRB.status.TIME_LIMIT:
        print("Time Limit Reached");
        LB = model.objVal;
        G_LB = model.objBound;
        print('Iteration: %g' %it); 
        print('LB (Master): %g' %LB);
        print('G_LB (Master): %g' %G_LB);
        for i,j in G.edges:
            gammaStar[i,j] = gamma[i,j].x;
            if gamma[i,j].x > 0:
                print('Interdict: ',i,j);
                 
        print('xBar_ts', 'flow :', xBar[t,s].x);
        print('x_ts', 'flow :', x[n][t,s].x);
        
        LB = G_LB;
    
    return gammaStar, xBar[t,s].x, LB, flow;
    
        

In [3]:
# Sub-problem

def Sub(gammaStar, xBar_ts, G, t, s, UB):
    
   
    M = A+1;
    
    xStar = {};
    zStar = {};
    
    sub = gp.Model("Sub-Problem");
    #sub.setParam('TimeLimit', t_lim);
    
    z = sub.addVars(G.edges, vtype=GRB.BINARY);
    x = sub.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    
    sub.addConstrs(gp.quicksum(x[v,u] for u in G.successors(v)) 
                 - gp.quicksum(x[u,v] for u in G.predecessors(v)) == 0 for v in G.nodes);
       
    for i,j in G.edges:
        sub.addConstr(x[i,j] <= G.edges[i,j]['capacity']*(1-gammaStar[i,j]));
        if G.edges[i,j]['special'] == 0:
            sub.addConstr(z[i,j] == 0);
        else:
            sub.addConstr(x[i,j] >= (1/M)*z[i,j]);
                
    sub.addConstr(x[t,s] - xBar_ts >= 0);
          
            
        
    sub.setObjective(gp.quicksum(z[i,j]*G.edges[i,j]['special'] for i,j in G.edges), GRB.MAXIMIZE);
    sub.update();
    sub.setParam("OutputFlag", 0);
    sub.optimize();
    
    
    if sub.status == GRB.status.OPTIMAL:
        objVal = sub.objVal;
        UB = objVal;
        print('UB (Sub): %g ' %UB);
        
        for i,j in G.edges:
            xStar[i,j] = x[i,j].x;
            zStar[i,j] = z[i,j].x;
        
        
        
    else:
        print ('No Solution!');

    
    return xStar, zStar, UB;
    

In [4]:
# Naive Approach to check optimality

def naive_approach(G, s, t, UB):
    
    UB_min = 10000;
    
    naive = gp.Model("OptimalityCheck");

    g = naive.addVars(G.edges, vtype=GRB.BINARY);
    x = naive.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);

    lst = list(itertools.product([0, 1], repeat=len(G.edges)));

    ctr_l = 0;
    for l in lst:
        ctr_e = 0;
        sum_g = 0;
        for i,j in G.edges:
            g[i,j] = l[ctr_e];
            ctr_e = ctr_e + 1;
            sum_g = sum_g + g[i,j]*G.edges[i,j]['cost'];
        
        if sum_g <= budget[b]:
            naive.remove(naive.getConstrs());
            naive.update();  
            naive.addConstrs(gp.quicksum(x[j,i] for i in G.successors(j)) 
                             - gp.quicksum(x[i,j] for i in G.predecessors(j)) == 0 for j in G.nodes);
            for i,j in G.edges:
                naive.addConstr(x[i,j] - (1-g[i,j])*G.edges[i,j]['capacity'] <= 0);
            
            naive.setObjective(x[t,s], GRB.MAXIMIZE);
            naive.setParam("OutputFlag", 0);
            naive.optimize();
            
            for i,j in G.edges:
                if g[i,j] > 0:
                    print('Interdict:', i, j);
            
            '''
            for i,j in G.edges:
                if x[i,j].x > 0.0001:
                    print('Flow', i, j, ':',x[i,j].x );
            '''
            
            obj = naive.objVal;
            
            x_prime, z_prime, UB_prime = Sub(g, x[t,s].x, G, t, s, UB);
            #print('UB (Naive - Sub):', UB_prime);
            
            if UB_min > UB_prime:
                UB_min = UB_prime;
            

    return UB_min;    
    

In [5]:
# Main

#'''
# Creating Results Files ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

file_summary = open("Results_Summary.csv", "w");

file_summary.write('Instance\t, Budget\t, Nodes\t, Arcs\t, OptGap\t,');
file_summary.write('Obj_value\t, Flow\t, Level_1\t, Level_2\t, Level_3\t, Other_Levels\t, Run_Time\n');
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#'''

num = 1;

m = {};
budget = [0,5,10,15];
rate = [1];

T_Lim = 3600;

network = {};
z = {};
xBar_ts = {};




# Input -- Reading the networks
for n in range(num):
    network[n] = 'Net'+str(n+1);
    
    
    for b in range(len(budget)):       
        
        for r in range(len(rate)):
            print('\n');
            print(network[n], ', budget', budget[b]);
            
            start_time = time.time(); 
            
            with open(network[n]+'.csv', newline='') as f:
                reader = csv.reader(f);
                row1 = next(reader);
                s = int(row1[0]);
                t = int(row1[1]);
                calA_level = int(row1[2]);
    
                G = nx.DiGraph();
                data = pd.read_csv(network[n]+'.csv',skiprows=1, header=None);
                n_edge = len(data.index+1);
        
                for i in range(n_edge): 
                    G.add_edge(data.iat[i,0], data.iat[i,1], capacity= data.iat[i,2], 
                    cost=data.iat[i,3], special=data.iat[i,4], level=data.iat[i,5]);
                    
                for i,j in G.edges:
                    if G.edges[i,j]['special'] == 1:
                        z[i,j] = 1;
                    else:
                        z[i,j] = 0;
                        
                                           
                    
                A = 0;
                U = G.edges[t,s]['capacity'];

                for i,j in G.edges:
                    if G.edges[i,j]['special'] == 1:
                        A = A + 1;
                        

            # Tau for the linearization of x_{ts}*delta
            tau = range(math.ceil(math.log(U))+1); #[];
                
                
        # C&CG       
            LB = 0;
            UB = 1e6;
            eps = 0.001;
            
            Z = [];
            Z.append(z);        
     
            Gamma = [];
            
            gammaStar = {};
            xBarStar = {};
            
            Flow = [];
            
            # Checking Optimality - Naive Approach
            '''
            if (G.number_of_edges() <= 20):
                print('Naive Approach');
                UB_min = naive_approach(G, s, t, UB);
                print('UB_min', UB_min);
            print('\n');
            '''
            
            
            it = 1;
            t_start = time.time();
                        
            while (UB - LB > eps and time.time() - t_start < T_Lim):           
                gammaStar, xBar_ts, LB, Maxflow = Master(Z, budget[b], G, tau, s, t, it, max(0,T_Lim - (time.time()-t_start)));
                Gamma.append(gammaStar);
                Flow.append(Maxflow);
                xStar, zStar, UB = Sub(gammaStar, xBar_ts, G, t, s, UB);
                Z.append(zStar);
                it = it + 1;
            
            
            obj = UB;
            sol = A - UB;
            



#'''  
# Printing results to file   ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            
            file = open('results_'+network[n]+'_b'+str(budget[b])+'.txt', "w");
            
            file.write('Number of Nodes: %g' % (t+1) +'\n');
            file.write('Number of Arcs: %g' % G.number_of_edges() +'\n');
            file.write('Budget: %g' % budget[b] +'\n\n');
            
            if (UB-LB) == 0:
                OptGap = 0.0;
                file.write('Solved to Optimality \n');
                file.write('Number of special arcs without flow: %g' % sol +'\n');
                file.write('Number of special arcs with flow: %g' % obj +'\n');
            else:
                file.write('Reached Time Limit \n');
                file.write('UB (Sub): %g' % UB +'\n'); 
                file.write('LB (MP): %g' % LB +'\n');
                OptGap = (UB-LB)/UB;
                file.write('Opt Gap: %s' % str("{:.3f}".format((OptGap)))+'\n');
            
            
            file.write('Max-Flow: %g' % Maxflow +'\n');
            file.write('Run-time: %s' % "{:.3f}".format((time.time() - start_time)) + ' sec');
            
            file.write('\n\n');    
            file.write('Level 0 Capacities: ' + str(G.edges[s,1]['capacity']) +'\n');
        
            
            lev1 = 0;
            lev2 = 0;
            lev3 = 0;
            other_level = 0;
            file.write('\nInterdiction plan: \n');
            
            key1 = False;
            key2 = False;
            key3 = False;
                
            for i, j in G.edges: 
                if gammaStar[i,j] > 0.00000001: 
                    if G.edges[i,j]['level'] == 1:
                        if key1 == False:
                            file.write('Level 1:' +'\n');
                            key1 = True;
                        file.write('Arc (%s,%s) ' %(i,j) +'\n');
                        lev1 += 1;
                    elif G.edges[i,j]['level'] == 2:
                        if key2 == False:
                            file.write('Level 2:' +'\n');
                            key2 = True;
                        file.write('Arc (%s,%s) ' %(i,j) +'\n');
                        lev2 += 1;
                    elif G.edges[i,j]['level'] == 3:
                        if key3 == False:
                            file.write('Level 3:' +'\n');
                            key3 = True;
                        file.write('Arc (%s,%s) ' %(i,j) +'\n');
                        lev3 += 1;
                    else:
                        file.write('Other Levels:' +'\n');
                        file.write('Arc (%s,%s) ' %(i,j) +'\n');
                        other_level += 1;
                   
                        
            
            file.write('\n');
            file.write('Special arcs with flow' +'\n');    
                
            for i, j in G.edges:
                if G.edges[i,j]['special'] == 1:
                    if xStar[i,j] > 0.0000001:
                        file.write('Arc (%s,%s): ' %(i,j) + '\t Flow: %f' % xStar[i,j] +'\n');
                        
            '''
            file.write('\nZ[i,j]:' +'\n');
            for i, j in G.edges:
                if G.edges[i,j]['special'] == 1:
                    if zStar[i,j] > 0.0000001: 
                        file.write('Arc (%s,%s): ' %(i,j) + '\t z: %f' % zStar[i,j] +'\n');
            '''
                
            file_summary.write(network[n]+'_b'+str(budget[b])+','+str(budget[b])+','+str(t+1)+','+str(G.number_of_edges())+','
                               +str("{:.3f}".format(OptGap))+','+str(obj)+','+str("{:.2f}".format(Maxflow))+','+str(lev1)+','+str(lev2)+','
                               +str(lev3)+','+str(other_level)+','+str("{:.3f}".format((time.time() - start_time)) + ' sec')+'\n');
            
            
            file.write('\n');
            file.close();

                
file_summary.close();              
#'''    



Net1 , budget 0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-30
Set parameter TimeLimit to value 3600
Set parameter IntegralityFocus to value 1
Set parameter NumericFocus to value 2
Iteration: 1
LB (Master): 16
xBar_ts flow : 0.0
x_ts flow : 377.0
UB (Sub): 16 


Net1 , budget 5
Set parameter TimeLimit to value 3600
Set parameter IntegralityFocus to value 1
Set parameter NumericFocus to value 2
Iteration: 1
LB (Master): 11
Interdict:  1 7
Interdict:  1 8
Interdict:  2 7
Interdict:  3 7
Interdict:  4 6
xBar_ts flow : 127.0
x_ts flow : 263.0
UB (Sub): 11 


Net1 , budget 10
Set parameter TimeLimit to value 3600
Set parameter IntegralityFocus to value 1
Set parameter NumericFocus to value 2
Iteration: 1
LB (Master): 6
Interdict:  1 8
Interdict:  2 6
Interdict:  2 7
Interdict:  2 8
Interdict:  3 5
Interdict:  3 6
Interdict:  3 8
Interdict:  4 6
Interdict:  4 7
Interdict:  4 8
xBar_ts flow : 124.0
x_ts flow : 139.0
UB (Sub): 6 


Net1 , budget 15